In [ ]:
from pytorch_lightning.loggers import NeptuneLogger

run = NeptuneLogger(
    project="andreasm/ufed",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI4NTVlOWQ1ZS0xMjQwLTRlZTktYjllMC0wZmI0ODBhYjA2MGMifQ==",
     log_model_checkpoints=False
)  # your credentials

In [ ]:
from network import FE_network
from model import Estimator
from dataset import Dataset
from torch.utils.data import DataLoader, random_split
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

In [ ]:
dataset = Dataset('./output_with_forces.csv', ['phi', 'psi'], ['F_phi', 'F_psi'])


# use_mean = False
weight_decay=0.0
spectral_norm = False  
learning_rate = 0.001

# updates = 100000
batch_size = 256
input_dim = dataset.cvs.shape[1]
width = 50
depths = 5
acti = 'Softplus' # 'Silu', 'Softplus'
valid_ratio = 0.2
spectral_norm = False
patience=20
max_epochs = 100


# mean = None
# std = None
# mean_force=None
# std_force = None
net  = FE_network(input_dim, width, depths, acti, spectral_norm) # mean, std, mean_force, std_force)

accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
model = Estimator(net, run, accelerator=accelerator, weight_decay=weight_decay, lr=learning_rate)
# if use_mean:
#     forces_pred = model.net.forward(torch.Tensor(dataset.cvs)).detach().cpu().numpy()
#     mean_force = torch.Tensor(forces_pred.mean(axis=0, keepdims=True))
#     std_force = torch.Tensor(forces_pred.std(axis=0, keepdims=True))
#     net.whitening_layer_model.set_both(mean_force, std_force)
#     net.whitening_layer_output.set_both(torch.Tensor(dataset.force_mean), torch.Tensor(dataset.force_std))

n_val = int(len(dataset)*valid_ratio)

train_data, val_data = random_split(dataset, [len(dataset) - n_val, n_val])

loader_train = DataLoader(train_data, batch_size=batch_size, shuffle=True, 
                            # num_workers=0, 
                            # prefetch_factor=2
                            )
loader_val = DataLoader(dataset, batch_size=len(dataset), shuffle=False)
# max_epochs = updates // len(loader_train) + 1
trainer = pl.Trainer(accelerator=accelerator, logger=model.neptune_logger, max_epochs=max_epochs)
# trainer.callbacks += [EarlyStopping(monitor="val_loss", mode="min", patience=patience)] 
trainer.fit(model, loader_train, loader_val)

In [ ]:
loader_train = DataLoader(train_data, batch_size=len(train_data), shuffle=True, 
                            # num_workers=0, 
                            # prefetch_factor=2
                            )
loader_val = DataLoader(val_data, batch_size=len(val_data), shuffle=False)
updates = 20
max_epochs = updates // len(loader_train) + 1
# model.set_learning_rate(0.01)
trainer = pl.Trainer(accelerator=accelerator, logger=model.neptune_logger, max_epochs=max_epochs)
# trainer.callbacks += [EarlyStopping(monitor="val_loss", mode="min", patience=patience)] 
trainer.fit(model, loader_train, loader_val)

In [ ]:
# plot free energy
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import griddata
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

# create grid on dihedral angles
phi = np.linspace(-np.pi, np.pi, 100)
psi = np.linspace(-np.pi, np.pi, 100)
phi, psi = np.meshgrid(phi, psi)
phi = phi.flatten()
psi = psi.flatten()
grid = np.vstack((phi, psi)).T
print(grid.shape)

In [ ]:
# compute free energy
with torch.no_grad():
    grid = torch.Tensor(grid)
    grid = grid.to(model.device)
    F = model.net.predict(grid)
    F = F.cpu().numpy()

In [ ]:
F = F.reshape(100, 100)
print(F.max()- F.min())

In [ ]:
plt.imshow(F, extent=[-np.pi, np.pi, -np.pi, np.pi], cmap=cm.viridis, origin='lower')